In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import stackstac
import planetary_computer as pc
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

# Import modular function
from env_function import environmental_variables

# Start Dask (safe + required)
cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit="8GB"
)
client = Client(cluster)

client

In [ ]:
bbox = (-82.7167, 27.5833, -82.3833, 28.0333)  # Tampa Bay
start_date = "2019-01-01"
end_date = "2024-12-31"

env_data = environmental_variables(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    variables=["sst", "precip"]
)

sst_lazy = env_data["sst"]          # xarray DataArray (lazy)
precip_items = env_data["precip"]   # list of STAC Items

type(sst_lazy), type(precip_items)


In [ ]:
import pandas as pd

# Refresh SAS tokens (REQUIRED)
for item in precip_items:
    pc.sign_inplace(item)

# Make start/end timezone-aware (UTC)
start = pd.Timestamp("2019-01-01", tz="UTC")
end = pd.Timestamp("2024-12-31", tz="UTC")

# Filter items strictly to 2019–2024
precip_filtered = [
    item for item in precip_items
    if start <= pd.to_datetime(item.datetime) <= end
]

len(precip_filtered)


In [ ]:
# Stack STAC items (NO chunks here)
precip_stack = stackstac.stack(
    precip_filtered,
    assets=["cog"],
    epsg=4326,
    fill_value=np.nan
)

# Apply chunking AFTER stacking (this is the correct pattern)
precip_stack = precip_stack.chunk({"time": 30})

# Monthly aggregation
precip_monthly = (
    precip_stack
    .mean(dim=["x", "y"])          # spatial mean
    .squeeze(drop=True)
    .resample(time="1ME")          # monthly
    .sum(min_count=1)              # precipitation sum
)


In [ ]:
final_ds = xr.Dataset({
    "sst": sst_lazy,
    "precip": precip_monthly
})

final_df = final_ds.compute().to_dataframe()
final_df.head()


In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(final_df.index, final_df["sst"], label="Monthly SST (°C)")
plt.plot(final_df.index, final_df["precip"], label="Monthly Precip (mm)")
plt.title("Monthly SST & Precipitation (2019–2024)")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- 1. Separate DataFrames ---

# Select the SST column and save it to a new DataFrame
sst_df = final_df[['sst']].copy()

# Select the Precipitation column and save it to a new DataFrame
precip_df = final_df[['precip']].copy()

# --- 2. Define File Paths ---
sst_path = "kalu_sst_data.csv"
precip_path = "kalu_precip_data.csv"

# --- 3. Save to CSV ---
sst_df.to_csv(sst_path, index=True, index_label='time')
print(f"✔ Successfully saved SST data to: {sst_path}")

precip_df.to_csv(precip_path, index=True, index_label='time')
print(f"✔ Successfully saved Precipitation data to: {precip_path}")

# --- 4. Cleanup Dask Resources (Crucial) ---
try:
    client.close()
    cluster.close()
    print("\nDask client and cluster successfully shut down.")
except Exception as e:
    print(f"Could not shut down Dask resources: {e}")

# Display the paths

In [1]:
from analysis import (
    load_env_timeseries,
    join_indices_and_env,
    compute_correlations_and_rmse,
)
from grace_functions import compute_wqi_indices

# 1) Define study box + time
bbox = (-82.7167, 27.5833, -82.3833, 28.0333)  # Tampa Bay

start_date = "2019-01-01"
end_date = "2022-12-31"  # match your WQI function default window

# 2) Grace's function: NDWI/NDTI/NDCI over Tampa Bay
wqi_df, wqi_rolling, wqi_monthly = compute_wqi_indices(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    export_csv=False,
    anomaly_detection=True,
    rolling_window=3,
)

# 3) Kalu's function: monthly SST/precip etc.
env_da = load_env_timeseries(bbox=bbox, start_date=start_date, end_date=end_date)

# 4) Join and compute stats
df = join_indices_and_env(wqi_df, env_da)
stats = compute_correlations_and_rmse(df)

stats


Cloud filtered: 500 → 195 scenes (<20.0% cloud cover)

=== FULL DATA QUALITY DIAGNOSTICS (First 5 Scenes) ===

SCENE 1: 2022-12-29 | ID: S2A_17RL...
Thumbnail: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/17/R/LL/2022/12/S2A_17RLL_20221229_0_L2A/thumbnail.jpg
Cloud cover (metadata): 7.814539%
SCL analysis failed: Reindexing only valid with uniquely valued Index objects
NDWI check failed: cannot access local variable 'scene' where it is not associated with a value
--------------------------------------------------------------------------------

SCENE 2: 2022-12-29 | ID: S2A_17RL...
Thumbnail: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/17/R/LM/2022/12/S2A_17RLM_20221229_0_L2A/thumbnail.jpg
Cloud cover (metadata): 8.301094%
SCL analysis failed: Reindexing only valid with uniquely valued Index objects
NDWI check failed: cannot access local variable 'scene' where it is not associated with a value
-------------------------------------------

RuntimeError: Error opening 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/17/R/LM/2021/10/S2B_17RLM_20211010_0_L2A/B08.tif': RasterioIOError('CURL error: Recv failure: Connection reset by peer')

In [6]:

# 1. Imports and study domain
from analysis import (
    load_env_timeseries,
    join_indices_and_env,
    compute_correlations_and_rmse,
    run_pca,            # correct function name
)
from grace_functions import compute_wqi_indices

# 1) Define study box + time
bbox = (-82.7167, 27.5833, -82.3833, 28.0333)  # Tampa Bay
start_date = "2019-01-01"
end_date = "2022-12-31"  # match WQI window


In [4]:
# 2. Monthly and seasonal environmental time series
# 2) Kalu's function: monthly SST / precip / other env variables
env_da = load_env_timeseries(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
)

env_da


Note: SST conversion (Kelvin to Celsius) applied.


<xarray.DataArray 'sst' (time: 24)> Size: 96B
dask.array<stack, shape=(24,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 192B 2019-01-31 2019-02-28 ... 2020-12-31
Attributes:
    depth:          20 cm
    long_name:      analysed sea surface temperature
    source:         ATSR<1,2>-ESACCI-L3U-v2.0, AATSR-ESACCI-L3U-v2.0, AVHRR<0...
    standard_name:  sea_water_temperature
    units:          kelvin
    valid_max:      4500
    valid_min:      -300

In [9]:
# 3. WQI indices (NDWI, NDTI, NDCI)
# 3) Grace's function: NDWI / NDTI / NDCI over Tampa Bay
wqi_df, wqi_rolling, wqi_monthly = compute_wqi_indices(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    export_csv=False,
    anomaly_detection=True,
    rolling_window=3,
)

wqi_df.head()


Cloud filtered: 500 → 195 scenes (<20.0% cloud cover)

=== FULL DATA QUALITY DIAGNOSTICS (First 5 Scenes) ===

SCENE 1: 2022-12-29 | ID: S2A_17RL...
Thumbnail: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/17/R/LL/2022/12/S2A_17RLL_20221229_0_L2A/thumbnail.jpg
Cloud cover (metadata): 7.814539%
SCL analysis failed: Reindexing only valid with uniquely valued Index objects
NDWI check failed: cannot access local variable 'scene' where it is not associated with a value
--------------------------------------------------------------------------------

SCENE 2: 2022-12-29 | ID: S2A_17RL...
Thumbnail: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/17/R/LM/2022/12/S2A_17RLM_20221229_0_L2A/thumbnail.jpg
Cloud cover (metadata): 8.301094%
SCL analysis failed: Reindexing only valid with uniquely valued Index objects
NDWI check failed: cannot access local variable 'scene' where it is not associated with a value
-------------------------------------------

RuntimeError: Error reading Window(col_off=897, row_off=0, width=897, height=37391) from 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/17/R/LL/2020/9/S2B_17RLL_20200905_1_L2A/B03.tif': RasterioIOError('Read failed. See previous exception for details.')

In [ ]:
wqi_monthly


In [ ]:
# 4. Join indices + environment and compute correlations / RMSE

# 4) Join WQI indices with environmental time series
df = join_indices_and_env(wqi_df, env_da)

# 5) Correlations and RMSE between WQI and env variables
stats = compute_correlations_and_rmse(df)
stats


In [ ]:
# 6) PCA on joined WQI–environment dataset
# 6) PCA on joined WQI–environment dataset
pca_results = run_pca(df, n_components=2)

loadings = pca_results["loadings"]
explained_variance = pca_results["explained_variance"]
variables = pca_results["variables"]
pcs = pca_results["pcs"]

loadings
